In [ ]:
import os
import kitchen.ingredients as k
import dropkick as dk
import numpy as np
import pandas as pd
import MILWRM.ST as st
import MILWRM.MxIF as mxif
import scanpy as sc
import sys; sys.path.append("../../../../../utility_scripts/")
from visium_utils import deconvolve_cnmf

# some stuff to make this notebook work better with Scanpy
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

---
# Read in key dataframe with sample information

In [ ]:
sample_key = pd.read_csv("../../../../resources/visium_sample_key.csv", index_col=0)

In [ ]:
# markers in COLON MAP PCA data
allmarkers = ["ACTININ","BCATENIN","CD3D","CD4_","CD8","CD11B","CD20","CD45","CD68",
              "CGA","COLLAGEN","COX2","DAPI","ERBB2","FOXP3","GAMMAACTIN","HLAA","LYSOZYME","MUC2",
              "NAKATPASE","OLFM4","SOX9","PANCK","PCNA","PEGFR","PSTAT3","SMA","SNA","VIMENTIN"]

eliotmarkers = ["VIMENTIN","SOX9","SMA","PSTAT3","PEGFR","PCNA","PANCK","OLFM4",
                "NAKATPASE","MUC5AC","MUC2","LYSOZYME","HLAA","GAMMAACTIN","FOXP3",
                "ERBB2","COLLAGEN","CGA","CDX2","CD68","CD45","CD20","CD11B","CD8",
                "CD4_","CD3D","BCATENIN","AQP5","COX2"]

channels_26sep22 = ["CEACAM5","DPEP1","CD27","AQP5","VIMENTIN","SOX9","SMA","PSTAT3","PEGFR","PCNA",
                    "PANCK","OLFM4","NAKATPASE","MUC5AC","MUC2","LYSOZYME","HLAA","GAMMAACTIN",
                    "FOXP3","COLLAGEN","CGA","CDX2","CD68","CD45","CD20","CD11B","CD8","CD4_",
                    "CD3D","BCATENIN"]

In [ ]:
# common channels among all slides
print(set(channels_26sep22).intersection(set(allmarkers).intersection(set(eliotmarkers))))

In [ ]:
for s in sample_key.index:
    if os.path.exists("IES_npz/{}_IES.npz".format(s)):
        print("Skipping {}".format(s))
    else:
        try:
            print("\nStarting: {}".format(s))
            print("Processing {}".format(sample_key.loc[s, "MxIF_npz"]))
            # read in img npz
            a_mx = mxif.img.from_npz("{}".format(sample_key.loc[s, "MxIF_npz"]))
            # read in unfiltered anndata
            a = sc.read(sample_key.loc[s, "unfiltered_adata"])
            # clean up anndatas so they're lightweight
            del a.var
            a = a[:,:5].copy()

            # read in master anndata
            a_cellprops = sc.read("../../" + sample_key.loc[s, "MILWRM_VUMCrefNMF30_adata"])
            # clean up anndatas so they're lightweight
            del a_cellprops.var
            a_cellprops = a_cellprops[:,:5].copy()

            # pre-align unfiltered anndata with master anndata
            a.obs["in_tissue"] = 0
            a.obs.loc[a_cellprops.obs_names, "in_tissue"] = 1
            # add MxIF to Visium anndata object
            a.uns["spatial"][sample_key.loc[s,"sample_key"]]["images"]["MxIF"] = np.dstack([a_mx.img, a_mx.mask])
            a.uns["spatial"][sample_key.loc[s,"sample_key"]]["scalefactors"]["tissue_MxIF_scalef"] = (
                a_mx.img.shape[1] / a.uns["spatial"][sample_key.loc[s,"sample_key"]]["images"]["hires"].shape[1]
            ) * a.uns["spatial"][sample_key.loc[s,"sample_key"]]["scalefactors"]["tissue_hires_scalef"]

            # trim MxIF to Visium coordinates
            a = st.map_pixels(a, img_key="MxIF", filter_label="in_tissue", map_size=a_mx.img.shape[:2])
            st.trim_image(a, channels=a_mx.ch+["tissue_mask"], plot_out=False)
    
            # read in master
            a_master = sc.read("../../datasets/master_anndatas/{}_master.h5ad".format(s))
            # transfer IES
            a.obs["IES"] = a_master.obs["IES"]
    
            p = st.assemble_pita(
                a,
                features=["IES"],
                use_rep=".obs",
                save_to=None,
                histo="hires",
                label=["IES"],
                cmap="plasma",
            )

            # save as MILWRM npz
            a_IES = mxif.img(p[0], channels=["IES"], mask=a_mx.mask)
            a_IES.to_npz("IES_npz/{}_IES.npz".format(s))

            print("Finished: {}".format(s))
        except:
            print("{} Failed".format(s))